<a href="https://colab.research.google.com/github/SirfTum/Project-3-LangChain-Function-Tool-Calling-Calculator/blob/main/Project_3_LangChain_Function_Tool_Calling_Calculator_ipynb3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain google-generativeai langchain_google_genai -q


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.5/41.5 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 7.5 MB/s eta 0:00:00


In [ ]:
from google.colab import userdata
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')

In [ ]:
from langchain_core.tools import tool
import requests

# we make this function as a wraper

@tool
# def multiply(a: int, b:int ) -> int:
#   """Multiply a and b """
#   print("function is called")
#   return a*b
def calculator(expression:str) -> float:
  """Evaluate a mathematical expression and return the result.

  Parameters:
  expression (str): Astring containging the mathematical expression to evaluate.

  Returns:
  float: The result of the evaluated expression.

  Examples:
  >>> evaluate_expression("2 + 3 * 4")
  14.0
  >>> evaluate_expression("(10/2)+8")
  13.0

  Note:
    - This function uses Python's 'eval()' to calculate the result.
    - Ensure the input is sanitized to avoid malicious code execution.
    """
  try:

    # Evaluate the expression safely

      result = eval(expression, {"__builtins__": {}})
      return float(result)
  except Exception as e:
      print(f"Error evaluating expression: {e}")
      return None





  # tool for latest news

import requests
@tool
def get_latest_news(topic: str) -> str:
  """
  Fetches the latest news for a given topic.
  Args:
    topic (str) : The topic to search for news articles.

  Returns:
    str: A fromatted string conaining the tool name, the latest news titles, and their respective links.

  Example:
    get_latest_news("Technology")
  """
  api_key = "9047c55c560840129435328dcf3ad307"
  url = f"https://newsapi.org/v2/everything?q={topic}&apiKey={api_key}"

  try:
    response = requests.get(url)
    data = response.json()

    if response.status_code == 200 and data.get('articles'):
      articles = data['articles']
      result = f"Tool used: get_latest_news\n get_latest_news tool is used \ nHere are the latest news articles"


      for article in articles[:10]:
        title = article['title']
        url = article['url']
        result += f"-{title}: {url}\n"


      return result
    else:
      return f"Error: Could not fetch news for {topic}. Reason: {data.get('message','Unknown error')}"
  except Exception as e:
    return f"Error: Unable to fetch news. Details : {str(e)}"



    # stock price
@tool
def get_stock_price(symbol: str) -> str:
  """Fetches the current stock price of a company based on its stock symbol using the Polygon API
  Args:
    symbol (str): The stock symbol of the company (e.g., 'AAPL' for Apple, 'GOOGLE' for Google).
  Returns:
    str: A message containing the current stock price of the company.

  Raises:
    HTTPError: If the HTTP request to the stock API fails (e.g., 404 or 500 status).
    RequestException: If there is an issue with the request itself (e.g., connection error).
    Exception: For any other unexpected errors during the execution of the function.
    """
  api_key = "rF2Rse0EEBnsfhyhL1HhGg5_61fJGh89"
  url = f"https://api.polygon.io/v2/aggs/ticker/{symbol}/prev"

  try:
    response = requests.get(url,params={"apiKey":api_key})
    response.raise_for_status()
    data = response.json()

    price=data.get('results',[{}])[0].get('c')

    if price:
      return f"Tool used:get_stock_price\n get_stock_price tool is used to find The current price of {symbol}"
    else:
      return f"Error: Could not retrieve stock data for {symbol}.\nTool used: get_stock_price"
  except requests.exceptions.HTTPError as http_err:
    return f"HTTP error occurred: {http_err}\nTool used: get_stock_price"
  except requests.exceptions.RequestException as req_err:
    return f"Request eror occured: {req_err}\nTool used: get_stock_price"
  except Exception as err:
    return f"An unexpected erorr occurred: {err}\nTool used: get_stock_price"






In [ ]:
# tools = [multiply]
tools = [calculator,get_latest_news, get_stock_price]

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model = 'gemini-2.0-flash-exp' , api_key=GOOGLE_API_KEY)

In [ ]:
from langchain.agents import initialize_agent , AgentType


In [ ]:
agent = initialize_agent(tools,llm , agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION)

In [ ]:
response = agent.invoke({"input":"what is 2 multiply by 3"})
response["output"]

'2 multiplied by 3 is 6.'

In [ ]:
response = agent.invoke({"input":"what is price of tesla"})
response["output"]

'I have already retrieved the stock price of Tesla.'